In [1]:
import keras

/Users/Takanori/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


本章で取り上げる内容<br>
・畳み込みNNの概要<br>
・過学習を抑制するためのデータ拡張<br>
・学習済みのCNNを使った特徴エンジニアリング<br>
・学習済みのCNNのファインチューニング<br>
・CNNが学習した内容と分類時の意思決定の可視化<br>

### 畳み込みニューラルネットワークの紹介

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # (out_depth, (window_height, window_width))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [4]:
# CNNの上に分類器を追加
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [6]:
# MNISTの画像でCNNを訓練
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
                         loss = 'categorical_crossentropy',
                        metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 48s 792us/step - loss: 0.1716 - acc: 0.9462
Epoch 2/5
60000/60000 [==============================] - 46s 765us/step - loss: 0.0490 - acc: 0.9848
Epoch 3/5
60000/60000 [==============================] - 44s 727us/step - loss: 0.0334 - acc: 0.9894
Epoch 4/5
60000/60000 [==============================] - 44s 736us/step - loss: 0.0249 - acc: 0.9922
Epoch 5/5
60000/60000 [==============================] - 43s 723us/step - loss: 0.0195 - acc: 0.9940


In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 3s 251us/step


In [8]:
test_acc

0.9907

### 畳み込み層のパラメータについて

・入力から抽出されたパッチのサイズ<br>
通常は3×3または５×5。ここの例では３×３が用いられている。<br>
・出力特徴マップの深さ<br>
畳み込みによって計算されるフィルタの数。<br>
深さは32で始まり、64で終わっている。<br>

### 最大値プーリングについて

最大値プーリング層を削除し、かなり大きな特徴マップを最後まで維持してもいいのでは？<br>

In [10]:
# モデルについて
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [11]:
model_no_max_pool.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


ダウンサンプリングを用いる理由<br>
1. 特徴マップの係数の数を減らす。<br>
2. 連続する畳み込みが調べるウィンドウを徐々に大きくすることで、空間フィルタ階層を抽出。<br>